In [6]:
!pip install langchain_community
!pip install langchain_openai
!pip install langchain
!pip install python-dotenv

In [2]:
%%writefile langchain-openai-luftairchatlog.txt
We are flying Lufthansa premium economy from Charlotte to Munich in Sept. 
Has anyone else flown with them in this class? 
If so can you review the experience. 
I paid $200 extra for each seat from Charlotte to Munich but had to pay $500 extra for each seat on the way home from Munich to Charlotte.
Not sure why this huge disparity but was essentially told by Lufthansa "we do so because we can". 
Thanks for your response in advance....


Overwriting langchain-openai-luftairchatlog.txt


In [3]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from dotenv import load_dotenv

In [5]:
load_dotenv()

raw_documents = TextLoader('langchain-openai-luftairchatlog.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=0,
                                      separator="\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever
)

query_list = [
    "What airline was being discussed?",
    "Which city did the flight depart from?",
    "Did they fly economy class?",
    "How much was paid for an extra seat from Charlotte?",
    "How much was paid for an extra seat from Munich?",
    "How much was paid for an extra seat from Cebu?",
    "Did the passenger enjoy her flight?"
    ]

for query in query_list:
    print(f"{qa.invoke(query)}\n")

{'query': 'What airline was being discussed?', 'result': ' Lufthansa'}

{'query': 'Which city did the flight depart from?', 'result': ' The flight departed from Charlotte.'}

{'query': 'Did they fly economy class?', 'result': ' No, they flew premium economy class.'}

{'query': 'How much was paid for an extra seat from Charlotte?', 'result': ' $200'}

{'query': 'How much was paid for an extra seat from Munich?', 'result': ' $500'}

{'query': 'How much was paid for an extra seat from Cebu?', 'result': " I don't have enough information to answer this question. The context given is about a flight from Charlotte to Munich and does not mention Cebu."}

{'query': 'Did the passenger enjoy her flight?', 'result': " I don't know."}

